### Machine limits for integer and floating point types
Get the machine limit for <i>float32</i>, <i>float64</i>, and <i>double</i> using <i>np.iinfo()</i>.

In [1]:
import numpy as np
np.iinfo(np.int32)

iinfo(min=-2147483648, max=2147483647, dtype=int32)

In [2]:
np.iinfo(np.int32).max

2147483647

In [3]:
np.iinfo(int)

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)

In [4]:
np.finfo(np.float32)

finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

In [5]:
np.finfo(np.float64)

finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)

In [6]:
np.finfo(np.double)

finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)

### Inexactness
Are the following expresions <i>True</i>?

In [7]:
0.1 + 0.1 + 0.1 == 0.3

False

In [8]:
x1 = 0.1
x2 = 0.10000000000000001
x3 = 0.1000000000000000055511151231257827021181583404541015625

In [9]:
eval(repr(x1)) == x1

True

In [10]:
eval(repr(x1)) == x2

True

In [11]:
eval(repr(x1)) == x3

True

### Holes in value range
What is the result of the following subtraction? Is it 0.0?

In [12]:
a = 1.0
b = 0.1
c = 1.1
c - a - b

8.326672684688674e-17

### Conversions

In [13]:
import numpy as np
np.__version__

'1.23.5'

Conversions to an integer can reveal the inaccuracies in a floating-point number. The closest *single-precision* floating-point number to 20.23 is slightly less than 20.23. When it is multiplied by a hundred, the result is slightly less than 2023.0. Note, there is no rounding in converting 'y' to an integer 'i', the number is truncated:

In [14]:
x = np.float32(20.23)
y = x * 100.
i = int(y)
print(i, y)

2022 2022.9999542236328


In [15]:
x = np.float64(20.23)
y = x * 100.
i = int(y)
print(i, y)

2023 2023.0


Assigning a single-precision number to a double-precision number doesn't increase the number of significant digits:

In [16]:
x = np.float32(1.66661)
y = np.float64(x)
print(y)

1.6666100025177002


Why there are simingly random <i>00025177002</i> numbers and not <i>00000000000</i>?

The floating point padding with zeros is done in the binary representation:
1.10101010101001101111010000000000000000000000000000000000010101...

### Rounding

In [17]:
round(256.49999) == 256

True

In [18]:
-1.225 * 100

-122.50000000000001

### Decimal fixed point and floating point arithmetic
https://docs.python.org/3/library/decimal.html

In [19]:
from decimal import *
getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [20]:
getcontext().prec = 6

In [21]:
Decimal(0.1) + Decimal(0.1) + Decimal(0.1)

Decimal('0.300000')

### Accuracy of floating point arithmetic
Examples from Donald E. Knuth The Art of Computer Programming, volume 2 / Seminumerical Algorithms, Section 4.2.2

In [22]:
from decimal import Decimal, getcontext
getcontext().prec = 8

u, v, w = Decimal(11111113), Decimal(-11111111), Decimal('7.51111111')
(u + v) + w

Decimal('9.5111111')

In [23]:
u + (v + w)

Decimal('10')

In [24]:
u, v, w = Decimal(20000), Decimal(-6), Decimal('6.0000003')
(u*v) + (u*w)

Decimal('0.01')

In [25]:
u * (v+w)

Decimal('0.0060000')

In [26]:
u, v, w = Decimal(8.0000001), Decimal(1.2500008), Decimal(8.0000008)
(u * v) * w

Decimal('80.000064')

In [27]:
u * (v * w)

Decimal('80.000057')

### Summing many numbers

In [28]:
import numpy as np

def summ():
    tenth = np.float32(0.1)
    count = np.float32(60*60*100*10)
    print(f"{count} {count*0.1}")
    sum = np.float32(0)
    n = np.int64(0)
    while n < 1000000:
        sum += tenth
        n += 1
        if n < 21 or n%36000 == 0:
            print(f"step {n} expected {0.1*n} solution {sum} diff {np.abs(0.1*n - sum)}")

summ()

3600000.0 360000.0
step 1 expected 0.1 solution 0.10000000149011612 diff 1.4901161138336505e-09
step 2 expected 0.2 solution 0.20000000298023224 diff 2.980232227667301e-09
step 3 expected 0.30000000000000004 solution 0.30000001192092896 diff 1.1920928910669204e-08
step 4 expected 0.4 solution 0.4000000059604645 diff 5.960464455334602e-09
step 5 expected 0.5 solution 0.5 diff 0.0
step 6 expected 0.6000000000000001 solution 0.6000000238418579 diff 2.3841857821338408e-08
step 7 expected 0.7000000000000001 solution 0.7000000476837158 diff 4.768371575369912e-08
step 8 expected 0.8 solution 0.8000000715255737 diff 7.152557368605983e-08
step 9 expected 0.9 solution 0.9000000953674316 diff 9.536743161842054e-08
step 10 expected 1.0 solution 1.0000001192092896 diff 1.1920928955078125e-07
step 11 expected 1.1 solution 1.1000001430511475 diff 1.4305114737211966e-07
step 12 expected 1.2000000000000002 solution 1.2000001668930054 diff 1.6689300519345807e-07
step 13 expected 1.3 solution 1.300000190

### Kahan summation algorithm
https://en.wikipedia.org/wiki/Kahan_summation_algorithm

In [29]:
import numpy as np

def kahan_summ():
    tenth = np.float32(0.1)
    count = np.float32(60*60*100*10)
    print(f"{count} {count*0.1}")
    sum = np.float32(0)
    n = np.int64(0)
    c = np.float32(0)
    while n < 1000000:
        y = tenth - c
        x = sum + y
        c = (x - sum) - y
        sum = x
        n += 1
        if n < 21 or n%36000 == 0:
            print(f"step {n} expected {0.1*n} solution {sum} diff {np.abs(0.1*n - sum)}")

kahan_summ()

3600000.0 360000.0
step 1 expected 0.1 solution 0.10000000149011612 diff 1.4901161138336505e-09
step 2 expected 0.2 solution 0.20000000298023224 diff 2.980232227667301e-09
step 3 expected 0.30000000000000004 solution 0.30000001192092896 diff 1.1920928910669204e-08
step 4 expected 0.4 solution 0.4000000059604645 diff 5.960464455334602e-09
step 5 expected 0.5 solution 0.5 diff 0.0
step 6 expected 0.6000000000000001 solution 0.6000000238418579 diff 2.3841857821338408e-08
step 7 expected 0.7000000000000001 solution 0.699999988079071 diff 1.1920929021691506e-08
step 8 expected 0.8 solution 0.800000011920929 diff 1.1920928910669204e-08
step 9 expected 0.9 solution 0.9000000357627869 diff 3.5762786843029915e-08
step 10 expected 1.0 solution 1.0 diff 0.0
step 11 expected 1.1 solution 1.100000023841858 diff 2.3841857821338408e-08
step 12 expected 1.2000000000000002 solution 1.2000000476837158 diff 4.7683715642676816e-08
step 13 expected 1.3 solution 1.3000000715255737 diff 7.152557368605983e-08

### Plot Example

In [30]:
import numpy as np

def plot_summ(k):
    s = np.float32(0)
    for i in range(k - 1):
        s += np.float32(0.1)
    return abs(k/10 - s)

for i in range(100000):
    if i%3600 == 0:
        print(plot_summ(i))

0.0
0.087249755859375
0.12872314453125
0.216552734375
0.304443359375
0.392333984375
0.343994140625
0.007568359375
0.359130859375
0.710693359375
1.062255859375
1.413818359375
1.76513671875
2.11669921875
2.46826171875
2.81982421875
3.17138671875
3.52294921875
3.87451171875
4.22607421875
4.57763671875
4.92919921875
5.28076171875
5.17578125
3.76953125
2.36328125
0.95703125
0.44921875


### Solar System

In [31]:
# solarsystem.py

import itertools
import math
import turtle
import numpy as np


# Solar System Bodies
class SolarSystemBody(turtle.Turtle):
    min_display_size = 20
    display_log_base = 1.1

    def __init__(
            self,
            solar_system,
            mass,
            position=(0, 0),
            velocity=(0, 0),
    ):
        super().__init__()
        self.mass = mass
        self.setposition(position)
        self.velocity = velocity
        self.display_size = max(
            math.log(self.mass, self.display_log_base),
            self.min_display_size,
        )

        self.penup()
        self.hideturtle()

        solar_system.add_body(self)

    def draw(self):
        self.clear()
        self.dot(self.display_size)

    def move(self):
        self.setx(self.xcor() + self.velocity[0])
        self.sety(self.ycor() + self.velocity[1])


class Sun(SolarSystemBody):
    def __init__(
            self,
            solar_system,
            mass,
            position=(0, 0),
            velocity=(0, 0),
    ):
        super().__init__(solar_system, mass, position, velocity)
        self.color("yellow")



class Planet(SolarSystemBody):
    colours = itertools.cycle(["red", "green", "blue"])

    def __init__(
            self,
            solar_system,
            mass,
            position=(0, 0),
            velocity=(0, 0),
    ):
        super().__init__(solar_system, mass, position, velocity)
        self.color(next(Planet.colours))


# Solar System
class SolarSystem:
    def __init__(self, width, height):
        self.solar_system = turtle.Screen()
        self.solar_system.tracer(0)
        self.solar_system.setup(width, height)
        self.solar_system.bgcolor("black")

        self.bodies = []

    def add_body(self, body):
        self.bodies.append(body)

    def remove_body(self, body):
        self.bodies.remove(body)

    def update_all(self):
        for body in self.bodies:
            body.move()
            body.draw()
        self.solar_system.update()
        
    @staticmethod
    def accelerate_due_to_gravity(
            first: SolarSystemBody,
            second: SolarSystemBody,
    ):
        force = first.mass * second.mass / first.distance(second) ** 2
        angle = first.towards(second)
        reverse = 1
        for body in first, second:
            acceleration = force / body.mass
            acc_x = acceleration * math.cos(math.radians(angle))
            acc_y = acceleration * math.sin(math.radians(angle))
            body.velocity = (
                body.velocity[0] + (reverse * acc_x),
                body.velocity[1] + (reverse * acc_y),
            )
            reverse = -1


In [32]:
# simple_solar_system.py

#from solarsystem import SolarSystem, Sun, Planet

solar_system = SolarSystem(width=1400, height=900)

sun = Sun(solar_system, mass=10_000)
planet = Planet(
    solar_system,
    mass=1,
    position=(-350, 0),
    velocity=(0, 5),
)

while True:
    solar_system.accelerate_due_to_gravity(sun, planet)
    solar_system.update_all()
    

TclError: invalid command name ".!canvas"